In [2]:
import pandas as pd

In [61]:
mydf = pd.read_csv("QTL_text_clean.csv")
mydf.head()

,combined,Category
0,Variance component analysis of quantitative tr...,1
1,"Single nucleotide polymorphism identification,...",0
2,Genetic resistance to Sarcocystis miescheriana...,0
3,Results of a whole-genome quantitative trait l...,1
4,Unexpected high polymorphism at the FABP4 gene...,0


In [ ]:
from together import Together

client = Together(api_key="99d9dd60650df75a56ad39665cb0d6a4aa9830edb2c850684e1aaf64e4c929d6")

In [ ]:
data = pd.read_csv('QTL_text_clean.csv')

augmented_data = []
for index, row in data.iterrows():
    print("At row : ", index)
    if row['Category'] == 1:
        while True:
            try:
                response = client.chat.completions.create(
                    model="deepseek-ai/DeepSeek-R1-Distill-Llama-70B-free",
                    messages=[
                        {
                                "role": "user",
                                "content": "I want you to generate five additional samples of the text provided to you. The text is the combination of the title and abstract of a relevant paper. Here, the relevant papers mean that these papers contains important geno-trait results that can be curated in the QTLdb. The samples generated should have the content of the entire text provided to you. I want you to ensure that each sample is as unique to each other as possible while prioritizing the keywords and the main idea of the text. Every sample should be printed in a single line and separated by a '~~' delimiter. The prompt output should not have any newline characters and contain only the line of samples and nothing else : " + row['combined']
                        }
                ],
                    max_tokens=None,
                    temperature=0.7,
                    top_p=0.7,
                    top_k=50,
                    repetition_penalty=1,
                    stop=["<｜end▁of▁sentence｜>"],
                    stream=True
                )

                prompt_resp = ""
                for chunk in response:
                    prompt_resp += (chunk.choices[0].delta.content or "")
            except:
                print(f"Failed for some reason, let's try {index} again")
            else:
                break

        prompt_resp = prompt_resp[prompt_resp.find("</think>")+8:]
        prompt_resp = prompt_resp.replace('\n', '')
        prompt_resp = prompt_resp.split('~~')
        for resp in prompt_resp:
            if len(resp)!=0:
                augmented_data.append({'combined': resp, 'Category': 1})
        

augmented_data = pd.DataFrame(augmented_data)
data = pd.concat([data, augmented_data], ignore_index=True)
print(data['Category'].value_counts())

At row :  0
At row :  1
At row :  2
At row :  3
At row :  4
At row :  5
At row :  6
At row :  7
At row :  8
At row :  9
At row :  10
At row :  11
At row :  12
At row :  13
At row :  14
At row :  15
At row :  16
At row :  17
At row :  18
At row :  19
At row :  20
At row :  21
At row :  22
At row :  23
At row :  24
At row :  25
At row :  26
At row :  27
At row :  28
At row :  29
At row :  30
At row :  31
At row :  32
At row :  33
At row :  34
At row :  35
At row :  36
At row :  37
At row :  38
At row :  39
At row :  40
At row :  41
At row :  42
At row :  43
At row :  44
At row :  45
At row :  46
At row :  47
At row :  48
At row :  49
At row :  50
At row :  51
At row :  52
At row :  53
At row :  54
At row :  55
At row :  56
At row :  57
At row :  58
At row :  59
At row :  60
At row :  61
At row :  62
At row :  63
At row :  64
At row :  65
At row :  66
At row :  67
At row :  68
At row :  69
At row :  70
At row :  71
At row :  72
At row :  73
At row :  74
At row :  75
At row :  76
At row : 

In [ ]:
print(data['combined'].str.len().min())
print(data['combined'].str.len().max())

# for every value in combined, find the value with minimum

2
6116


Category
0    10271
1     6457
Name: count, dtype: int64

In [92]:
# for every value in the combined column, remove extra space at the beginning and end of the string
aug_data = data.copy()
aug_data['combined'] = data['combined'].str.strip()
aug_data.head()

,combined,Category
0,Variance component analysis of quantitative tr...,1
1,"Single nucleotide polymorphism identification,...",0
2,Genetic resistance to Sarcocystis miescheriana...,0
3,Results of a whole-genome quantitative trait l...,1
4,Unexpected high polymorphism at the FABP4 gene...,0


In [93]:
# rename combined to text
aug_data.rename(columns={'combined': 'text'}, inplace=True)

aug_data = aug_data[aug_data['text'].str.len() >= 250]

aug_data.to_csv('augmented_data.csv', index=False)

In [94]:
aug_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15277 entries, 0 to 16727
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   text      15277 non-null  object
 1   Category  15277 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 358.1+ KB
